## Setup

Load libraries

In [1]:
import sys

sys.path.append('..')

In [2]:
import pickle
from snowmodels.utils import DefaultTuner

In [3]:
with open('../data/data_splits.pkl', 'rb') as f:
    data_splits = pickle.load(f)

## Default Hyperparameters

We'll start by running all four models using their default hyperparamaters. This is essential to see if there is any performance gains from tuning.

In [4]:
X_train = data_splits['X_train']
X_val = data_splits['X_val']
y_train = data_splits['y_train']
y_val = data_splits['y_val']

In [7]:
default_tuner = DefaultTuner(
        X_train=X_train,
        X_val=X_val,
        y_train=y_train,
        y_val=y_val
    )

baseline_results = default_tuner.run_default_models()


Running baseline models with default parameters...
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.008038 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 738
[LightGBM] [Info] Number of data points in the train set: 1905792, number of used features: 9
[LightGBM] [Info] Start training from score 0.302447
Training until validation scores don't improve for 50 rounds
Did not meet early stopping. Best iteration is:
[1000]	train's rmse: 0.0483899	valid's rmse: 0.0488552

Results with onehot encoder:
LightGBM - R²: 0.7199, RMSE: 0.0489, Best iteration: 1000
XGBoost - R²: 0.7362, RMSE: 0.0474, Best iteration: 999
ExtraTrees - R²: 0.6734, RMSE: 0.0528
RandomForest - R²: 0.7430, RMSE: 0.0468
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.004291 seconds.
You can set `force_row_wise=true` to remove the over

## Naive Optimization

The purpose of this optimization is to have an idea of how far I can go with tree addition to Random Forest and Extra Trees before running out of memory using their respective default configurations. This will allow me know the number of trees to set in optuna.